In [1]:
import requests
import pandas as pd
import json
import datetime

In [2]:
bitcoin_price_url = 'https://api.polygon.io/v2/aggs/ticker/X:BTCUSD/range/1/day/2022-01-01/2023-06-30?apiKey=maEQwPe36ret4bYTHkL8D4Tri4oAFLm4'

In [3]:
bitcoin_response = requests.get(bitcoin_price_url)

In [4]:
print(bitcoin_response)

<Response [200]>


In [5]:
bitcoin_response_content = bitcoin_response.content

In [6]:
bitcoin_data = bitcoin_response.json()
bitcoin_data

{'ticker': 'X:BTCUSD',
 'queryCount': 546,
 'resultsCount': 546,
 'adjusted': True,
 'results': [{'v': 14516.208110053272,
   'vw': 47107.5183,
   'o': 46211.24,
   'c': 47733.43,
   'h': 47967.12,
   'l': 46150,
   't': 1640995200000,
   'n': 465519},
  {'v': 10724.09386026835,
   'vw': 47243.3904,
   'o': 47746,
   'c': 47299.07,
   'h': 48012,
   'l': 46633.36,
   't': 1641081600000,
   'n': 431851},
  {'v': 16693.028504542646,
   'vw': 46630.0499,
   'o': 47297.66,
   'c': 46459.56,
   'h': 47586.58,
   'l': 45689,
   't': 1641168000000,
   'n': 450817},
  {'v': 24161.2826550835,
   'vw': 46464.56,
   'o': 46459.57,
   'c': 45814.61,
   'h': 47671.86280513,
   'l': 45515,
   't': 1641254400000,
   'n': 556220},
  {'v': 39467.84985036008,
   'vw': 45006.006,
   'o': 45817.13,
   'c': 43436.04,
   'h': 47076.55,
   'l': 42413.59,
   't': 1641340800000,
   'n': 798720},
  {'v': 31429.986673682462,
   'vw': 43078.5828,
   'o': 43457,
   'c': 43083.76,
   'h': 43794.5,
   'l': 42412,
  

In [7]:
bitcoin_df = pd.json_normalize(bitcoin_data['results'])
bitcoin_df

,v,vw,o,c,h,l,t,n
0,14516.208110,47107.5183,46211.24,47733.43,47967.120000,46150.00,1640995200000,465519
1,10724.093860,47243.3904,47746.00,47299.07,48012.000000,46633.36,1641081600000,431851
2,16693.028505,46630.0499,47297.66,46459.56,47586.580000,45689.00,1641168000000,450817
3,24161.282655,46464.5600,46459.57,45814.61,47671.862805,45515.00,1641254400000,556220
4,39467.849850,45006.0060,45817.13,43436.04,47076.550000,42413.59,1641340800000,798720
...,...,...,...,...,...,...,...,...
541,16345.892786,30276.9845,30480.24,30264.20,30685.000000,29900.00,1687737600000,358429
542,16409.937259,30623.5859,30272.24,30697.38,31020.540000,30223.60,1687824000000,346245
543,13861.890733,30246.1483,30696.06,30074.94,30731.000000,29834.20,1687910400000,364904
544,16071.586504,30503.4538,30070.46,30450.60,30849.000000,30036.18,1687996800000,348932


In [8]:
Date_Time_btc = pd.to_datetime(bitcoin_df.t, unit='ms').dropna()
Date_Time_btc

0     2022-01-01
1     2022-01-02
2     2022-01-03
3     2022-01-04
4     2022-01-05
         ...    
541   2023-06-26
542   2023-06-27
543   2023-06-28
544   2023-06-29
545   2023-06-30
Name: t, Length: 546, dtype: datetime64[ns]

In [9]:
bitcoin_df = bitcoin_df.drop(columns=['t'])

In [10]:
bitcoin_df_dt = pd.concat([bitcoin_df, Date_Time_btc], axis=1, join='inner')

In [11]:
bitcoin_df_dt = bitcoin_df_dt.rename(columns={
    'v': 'Volume',
    'vw': 'Volume Weight',
    'o': 'Open',
    'c': 'Close',
    'h': 'High',
    'l': 'Low',
    'n': 'Transactions',
    't': 'Date'
})
bitcoin_df_dt = bitcoin_df_dt.set_index(['Date'])
bitcoin_df_dt.head()

,Volume,Volume Weight,Open,Close,High,Low,Transactions
Date,,,,,,,
2022-01-01,14516.208110,47107.5183,46211.24,47733.43,47967.120000,46150.00,465519
2022-01-02,10724.093860,47243.3904,47746.00,47299.07,48012.000000,46633.36,431851
2022-01-03,16693.028505,46630.0499,47297.66,46459.56,47586.580000,45689.00,450817
2022-01-04,24161.282655,46464.5600,46459.57,45814.61,47671.862805,45515.00,556220
2022-01-05,39467.849850,45006.0060,45817.13,43436.04,47076.550000,42413.59,798720


In [12]:
bitcoin_df_dt.to_csv('btc_price_data.csv')